In [73]:
import json
import openai
import pickle


model_id = "gpt-4"
API_key = ""
openai.api_key=API_key

path_to_data = "/submission_X"
f = open(path_to_data)
data = json.load(f)
passages_response = {}

with open("response_validation_turns", 'r') as f:
    data_turns = f.readlines()

selected_turns = [elem.strip() for elem in data_turns]

for turn in data["turns"]:
    for response in turn["responses"]:
        turn_id = turn["turn_id"]
        if turn_id in selected_turns:
            if (response["rank"] == 1) or (response["rank"] == "1"):
                passages_response[turn_id] = {}
                res_text = response["text"]
                passages_response[turn_id]["response"] = res_text
                passages_response[turn_id]["documents"] = []
                for passage in response["passage_provenance"]:
                    if "used" in passage:
                        if passage["used"] == True:
                            print("yesss")
                            passages_response[turn_id]["documents"].append(passage["text"])



In [ ]:
def chatgpt_conversation(conversation_Log):
  response = openai.ChatCompletion.create(
      model = model_id,
      messages = conversation_Log
  )
  conversation_Log.append({'role':response.choices[0].message.role ,'content':response.choices[0].message.content.strip()})
  return conversation_Log

def run_one_sample(init_prompt):

    conversations = []
    conversations.append({'role': 'user', 'content': init_prompt})
    conversations = chatgpt_conversation(conversations)
    answer_r_prime = conversations[-1]['content'].strip()
    print(init_prompt)
    print("********************")
    print(answer_r_prime)
    print("**************************************")
    return answer_r_prime



In [90]:
passages_response = {}
for turn in data["turns"]:
    for response in turn["responses"]:
        turn_id = turn["turn_id"]
        if turn_id in selected_turns:
            if (response["rank"] == 1) or (response["rank"] == "1"):
                passages_response[turn_id] = {}
                res_text = response["text"]
                passages_response[turn_id]["response"] = res_text
                passages_response[turn_id]["documents"] = []
                newlist = sorted(response["passage_provenance"], key=lambda d: float(d['score']), reverse=True)
                print([float(d['score']) for d in newlist[0:5]])
                for passage in newlist[0:10]:
                    passages_response[turn_id]["documents"].append(passage["text"])

[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996, 0.995]
[0.999, 0.998, 0.997, 0.996,

In [108]:
with open('passages_response.pickle', 'rb') as handle:
    passages_response = pickle.load(handle)

In [109]:
len(passages_response)

73

In [110]:
prompts_per_turn = {}


for turn_id in passages_response:
    prompt_2 = ""
    if len(passages_response[turn_id]["response"])>10 and len(passages_response[turn_id]["documents"])>0:
        for doc_id, doc in enumerate(passages_response[turn_id]["documents"]):
            prompt_2 += "document "+str(doc_id+1) +": "+ doc.strip() + "\n"

        prompt_2 += "\nResponse: " + passages_response[turn_id]["response"] + "\n"

        prompts_per_turn[turn_id] = prompt_2

prompt_1 = """I will give you a text and a set of documents. I want to know if the text is generated from the given documents. You should tell me to what extent the text is grounded to the documents by selecting one of the given choices.\n\n"""

prompt_3 = """1. Does the text include any inforation that is not mentioned in the given documents? Please explain your answer.
2. Answer the previous question with yes or no."""

prompts_arr = []

for turn_id in prompts_per_turn:
    prompt_tmp = prompt_1 + prompts_per_turn[turn_id] + prompt_3
    prompts_arr.append([turn_id, prompt_tmp])


print(len(prompts_arr))


answers = {}

73


In [111]:
len(answers)

0

In [ ]:
prompts_arr

In [ ]:
for i in range(0, len(prompts_arr)):
    turn_id = prompts_arr[i][0]
    prompt = prompts_arr[i][1]
    print("index is: ", i)
    answer = run_one_sample(prompt)
    answers[turn_id] = answer
    print(turn_id)
    print("*******************************************************")

In [115]:
with open('llm-judgment-on-groundedness-of-submission_X.pickle',  'wb') as handle:
    pickle.dump(answers, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [116]:
X = []

for turn_id in answers:
    answer = answers[turn_id]
    tmp_ans = answer.split("\n")[-1]
    X.append(tmp_ans.split("2.")[1].strip().rstrip("."))

In [119]:
print(X.count('No'))
print(X.count('Yes'))

49
24


In [120]:
print(X)

['No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No']
